<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://emdros.org" target="_blank"><img align="left" src="files/images/Emdros-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="right" src="images/VU-ETCBC-xsmall.png"/></a>

# נָתַן and locatives



In [1]:
import sys
import collections
import subprocess

from lxml import etree

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.mql import MQL
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.4.7
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



In [2]:
API = fabric.load('etcbc4b', '--', 'ntn', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        oid otype monads number
        language lex g_word
        sp gn nu ps vt vs
        function typ det
        book chapter verse label
    ''',''),
    "prepare": prepare,
    "primary": False,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))
Q = MQL(API)

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.00s INFO: USING DATA COMPILED AT: 2015-05-04T13-46-20
  0.00s DETAIL: COMPILING a: UP TO DATE
  0.01s DETAIL: load main: G.node_anchor_min
  0.09s DETAIL: load main: G.node_anchor_max
  0.15s DETAIL: load main: G.node_sort
  0.20s DETAIL: load main: G.node_sort_inv
  0.63s DETAIL: load main: G.edges_from
  0.70s DETAIL: load main: G.edges_to
  0.78s DETAIL: load main: F.etcbc4_db_monads [node] 
  1.54s DETAIL: load main: F.etcbc4_db_oid [node] 
  2.25s DETAIL: load main: F.etcbc4_db_otype [node] 
  2.93s DETAIL: load main: F.etcbc4_ft_det [node] 
  3.17s DETAIL: load main: F.etcbc4_ft_function [node] 
  3.29s DETAIL: load main: F.etcbc4_ft_g_word [node] 
  3.53s DETAIL: load main: F.etcbc4_ft_gn [node] 
  3.70s DETAIL: load main: F.etcbc4_ft_language [node] 
  3.91s DETAIL: load main: F.etcbc4_ft_lex [node] 
  4.11s DETAIL: load main: F.etcbc4_ft_nu [node] 
  4.31s DETAIL: load main: F.etcbc4_ft_number [n

In [23]:
test_query = '''
select all objects where
[book book = Genesis
[chapter chapter = 45
[verse verse = 22
[sentence number = 73
[clause number = 1
    [phrase focus first and last] or [phrase focus first] [[phrase focus][gap?]]* [phrase focus last]
]
]]]]
'''
test_sheaf = Q.mql(test_query)
print('{} results'.format(test_sheaf.nresults()))
for x in test_sheaf.results():
    print(x)

1 results
((1368087, ((1368170, ((1415522, ((1129535, ((431935, ((620998,), (621000,))),)),)),)),)),)


In [3]:
ntn_sim_query = '''
select all objects where

[book[chapter[verse
[clause
 [phrase function = Pred 
     [word focus first (language = Hebrew and (lex = 'NTN[' or lex = 'FJM[')) ]
 ]
]
]]]
'''
clause_query = '''
select all objects where
[clause
    [word focus first and last] or [word focus first] [[word focus][gap?]]* [word focus last]
]
'''
clause_p_query = '''
select all objects where
[clause
    [phrase focus]
]
'''

In [4]:
sheaf_ntn_sim = Q.mql(ntn_sim_query)
print('{} results'.format(sheaf_ntn_sim.nresults()))
for x in sheaf_ntn_sim.results():
    print(x)
    break

1936 results
((1368087, ((1368126, ((1414158, ((426628, ((604966, ((314,),)),)),)),)),)),)


In [5]:
sheaf_clause = Q.mql(clause_query)
print('{} results'.format(sheaf_clause.nresults()))

87900 results


In [6]:
sheaf_clause_p = Q.mql(clause_p_query)
print('{} results'.format(sheaf_clause_p.nresults()))

253214 results


In [7]:
verb_index = {}
phrase_index = {}
word_index = {}
book_index = {}
passage_index = {}
relevant_clauses = set()
doubles = 0
for ((book, ((chapter, ((verse, ((clause, ((phrase, ((word,),)),)),)),)),)),) in sheaf_ntn_sim.results():
    if clause in relevant_clauses: doubles += 1
    relevant_clauses.add(clause)
    book_index[clause] = F.book.v(book)
    passage_index[clause] = '{} {}:{}#{}'.format(
        F.book.v(book), 
        F.chapter.v(chapter), 
        F.verse.v(verse), 
        clause,
    )
    wlex = F.lex.v(word)
    phrase_index[phrase] = wlex
    word_index[word] = wlex
nclauses = len(relevant_clauses)
print('{} clauses of which {} duplicates => total {}'.format(
    nclauses, doubles, nclauses - doubles,
))
print('{} phrases in index\n{} words in index'.format(len(phrase_index), len(word_index)))

1936 clauses of which 0 duplicates => total 1936
1936 phrases in index
1936 words in index


# Notes

Pronominal suffixes are not taken into account, but they do carry gender/number information!

In [9]:
outf = outfile('ntn_sim.csv')
nwritten = 0
tclauses = 0
clauses_w = set()
still_in_pre = 0
for ((clause, words),) in sheaf_clause.results():
    tclauses += 1
    clauses_w.add(clause)
    if clause not in relevant_clauses: continue
    pre_info = set()
    post_info = set()
    verb_info = ''
    in_pre = True
    for word in words:
        word = word[0]
        this_verb_info = word_index.get(word, None)
        if this_verb_info != None:
            in_pre = False
            verb_info += this_verb_info
        else:
            info = {
                F.lex.v(word).replace('_','-'),
                F.sp.v(word), 
                F.gn.v(word), 
                F.nu.v(word), 
                F.ps.v(word), 
                F.vt.v(word), 
                F.vs.v(word),
            } - {
                'NA', 
                'unknown',
            }
            if in_pre:
                pre_info |= info
            else:
                post_info |= info
    if in_pre: still_in_pre += 1
    outf.write('{},{},{},{}\n'.format(
        passage_index[clause], 
        verb_info, 
        '_'.join(pre_info | post_info), 
        book_index[clause],
    ))
    nwritten += 1
outf.close()
print('{} clauses, {} lines written'.format(tclauses, nwritten))
print('{} still in pre'.format(still_in_pre))

87900 clauses, 1936 lines written
0 still in pre


In [11]:
outfp = outfile('ntn_sim_p.csv')
outfpo = outfile('ntn_sim_po.csv')
nwritten = 0
tclauses = 0
clauses_p = set()
still_in_pre = 0
curclause = None
i = 0
for ((clause, ((phrase,),)),) in sheaf_clause_p.results():
    if clause != curclause:
        if curclause != None:
            if in_pre: still_in_pre += 1
            outfp.write('{},{},{}\n'.format(passage_index[curclause], verb_info, '_'.join(pre_info | post_info)))
            outfpo.write('{},{},{}\n'.format(passage_index[curclause], verb_info, '_'.join(pre_info_o | post_info_o)))
            nwritten += 1
        tclauses += 1
        clauses_p.add(clause)
        if clause not in relevant_clauses:
            curclause = None
            continue
        curclause = clause
        pre_info = set()
        pre_info_o = set()
        post_info = set()
        post_info_o = set()
        verb_info = ''
        in_pre = True
        i = 0
    else:
        i += 1
    this_verb_info = phrase_index.get(phrase, None)
    if this_verb_info != None:
        in_pre = False
        verb_info += this_verb_info
    else:
        info = {
            F.function.v(phrase), 
            F.typ.v(phrase), 
            F.det.v(phrase),
        } - {
            'NA', 
            'unknown',
        }
        info_o = {'{}{}'.format(i, v) for v in info}
        if in_pre:
            pre_info |= info
            pre_info_o |= info_o
        else:
            post_info |= info
            post_info_o |= info_o
if curclause != None:
    if in_pre: still_in_pre += 1
    outfp.write('{},{},{}\n'.format(passage_index[curclause], verb_info, '_'.join(pre_info | post_info)))
    outfpo.write('{},{},{}\n'.format(passage_index[curclause], verb_info, '_'.join(pre_info_o | post_info_o)))
    nwritten += 1
outfp.close()
outfpo.close()
print('{} clauses, {} lines written'.format(tclauses, nwritten))
print('{} still in pre'.format(still_in_pre))

247527 clauses, 1936 lines written
0 still in pre
